In [1]:
import numpy as np
import logging
import uuid

from scrs import Simage
from scrs.constants import OUT_DIR

In [2]:
IMG_SIZE = (9000, 9000)

logging.basicConfig()
logging.getLogger().setLevel(logging.WARNING)

logger = logging.getLogger()

In [3]:
NUM_RUNS = 100
SAT_LEVEL = 65535
READOUT_OFFSET = 0
NUM_STARS = 1000
MAX_SIGMA = 10
ALLOW_SATURATION = True
TYPE = "stars"

# Generate seeds in advance, before setting globally.
SEEDS = [np.random.randint(0, 100_000) for _ in range(NUM_RUNS)]

for i in range(NUM_RUNS):
    file_uuid = uuid.uuid4()

    print(f"Generating {i+1}/{NUM_RUNS}. UUID: {file_uuid}", end="\r" if i < NUM_RUNS - 1 else "\n")

    rng = np.random.default_rng(SEEDS[i])

    img: Simage = Simage().set_rng(rng).set_data(np.zeros(IMG_SIZE, dtype=np.uint16))

    if TYPE == "spectra":
        raise NotImplementedError("Spectra not implemented yet.")
        # img.add_spectra(...)
    else:
        img.add_stars(
            num_stars=NUM_STARS,
            max_sigma=MAX_SIGMA,
            sat_level=SAT_LEVEL,
            allow_sat=ALLOW_SATURATION,
        )

    (
        img
        # Enable history, starting with raw star image (no noise or cosmics)
        .enable_history(OUT_DIR / "auto" / "raw" / f"{file_uuid}_raw.fits")  # idx 2
        .add_noise()  # idx 1
        .add_cosmics(conversion_factor=0.001)  # idx 0
        .add_to_header(
            TYPE=TYPE,
            SATLEVEL=SAT_LEVEL,
            NUMSTARS=NUM_STARS,
            MAXSIG=MAX_SIGMA,
            ALLOWSAT=ALLOW_SATURATION,
            SEED=SEEDS[i],
        )
    )

Generating 100/100. UUID: 2321e4ca-21e4-44d1-a3c1-e16fd4998adf
